# 比例ハザードモデルの妥当性評価
モデルに基づく推測は、当てはめた統計モデルに完全に依存する。したがって、この推測が"妥当"であることを保証するためには、当てはめたモデルが、データに基づいて妥当であるということを要約して説明しなければならない。

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
# Variable          Description                     Codes / Values
# id                Identification Code             1 - 500
# age               Age at Hospital Admission       Years
# gender            Gender                          0 = Male, 1 = Female
# hr                Initial Heart Rate              Beats per minute
# sysbp             Initial Systolic Blood          mmHg
#                     Pressure
# diasbp            Initial Diastolic Blood         mmHg
#                     Pressure
# bmi               Body Mass Index                 kg/m^2
# cvd               History of Cardiovascular       0 = No, 1 = Yes
#                     Disease
# afb               Atrial Fibrillation             0 = No, 1 = Yes
# sho               Cardiogenic Shock               0 = No, 1 = Yes
# chf               Congestive Heart                0 = No, 1 = Yes
#                      Complications
# av3               Complete Heart Block            0 = No, 1 = Yes
# miord             MI Order                        0 = First, 1 = Recurrent
# mitype            MI Type                         0 = non Q-wave, 1 = Q-wave
# year              Cohort Year                     1 = 1997, 2 = 1999, 3 = 2001
# admitdate         Hospital Admission Date         mm/dd/yyyy
# disdate           Hospital Discharge Date         mm/dd/yyyy
# fdate             Date of last Follow Up          mm/dd/yyyy
# los               Length of Hospital Stay         Days between Hospital
#                                                     Discharge and Hospital
#                                                     Admission
# dstat             Discharge Status from           0 = Alive, 1 = Dead
#                     Hospital
# lenfol            Total Length of Follow-up       Days between Date of Last
#                                                     Follow-up and Hospital
#                                                     Admission Date
# fstat             Vital Status at Last            0 = Alive 1 = Dead
#                      Follow-up

WHAS500_SURVIVAL_DATASET_PATH = 'dataset/whas500.dat'
COLUMNS = [
    'age', 'gender', 'hr', 'sysbp', 'diasbp', 'bmi', 'cvd', 'afb', 'sho', 'chf', 'av3', 
    'miord', 'mitype', 'year', 'admitdate', 'disdate', 'fdate', 'los', 'dstat', 'lenfol', 'fstat'
]

whas500_df = pd.read_csv(WHAS500_SURVIVAL_DATASET_PATH, sep='\s+', header=None, index_col=0, names=COLUMNS)
whas500_df.head()

,age,gender,hr,sysbp,diasbp,bmi,cvd,afb,sho,chf,...,miord,mitype,year,admitdate,disdate,fdate,los,dstat,lenfol,fstat
1,83,0,89,152,78,25.54051,1,1,0,0,...,1,0,1,01/13/1997,01/18/1997,12/31/2002,5,0,2178,0
2,49,0,84,120,60,24.02398,1,0,0,0,...,0,1,1,01/19/1997,01/24/1997,12/31/2002,5,0,2172,0
3,70,1,83,147,88,22.14290,0,0,0,0,...,0,1,1,01/01/1997,01/06/1997,12/31/2002,5,0,2190,0
4,70,0,65,123,76,26.63187,1,0,0,1,...,0,1,1,02/17/1997,02/27/1997,12/11/1997,10,0,297,1
5,70,0,63,135,85,24.41255,1,0,0,0,...,0,1,1,03/01/1997,03/07/1997,12/31/2002,6,0,2131,0


## 1. 残差

 - $p$個の共変量をもつ$n$個の独立な時間の観測値があるとする
 - 時間の観測値、共変量および打ち切りお指示変数を$(t_{i}, x_{i}, c_{i})$の組で表すとする
     - ただし、$i=1,2,\cdots,n$であり、$c_{i} = 1$を打ち切りを受けなかった観測値とし、$c_{i}=0$を打ち切りを受けた観測値とする

### Schoenfeld残差(Schoenfeld residuals)
Schoenfeld残差は対数部分尤度の偏導関数に対する各被験者の寄与に基づいた残差である。

$k$番目の共変量に対する対数部分尤度の偏導関数は、
$$
\frac { \partial \log { { L }_{ p }\left( \beta  \right)  }  }{ \partial \beta _{ k } } =\sum _{ i=1 }^{ n }{ c_{ i }\left\{ x_{ ik }-\frac { \sum _{ j\in R\left( t_{ i } \right)  }{ x_{ jk }e^{ x_{ \prime  }^{ j }\beta  } }  }{ \sum _{ j\in R\left( t_{ i } \right)  }{ e^{ x_{ \prime  }^{ j }\beta  } }  }  \right\}  } =\sum _{ i=1 }^{ n }{ c_{ i }\left\{ x_{ ik }-{ \bar { x }  }_{ w_{ i }k } \right\}  } \qquad (5.1)
$$
である。ここで、
$$
{ \bar { x }  }_{ w_{ i }k }=\frac { \sum _{ j\in R\left( t_{ i } \right)  }{ x_{ jk }e^{ x_{ \prime  }^{ j }\beta  } }  }{ \sum _{ j\in R\left( t_{ i } \right)  }{ e^{ x_{ \prime  }^{ j }\beta  } }  } \qquad (5.2)
$$
である。$k$番目の共変量に対する$i$番目のデータのSchoenfeld残差の推定量は、

> $$
\hat { r_{ ik } } ={ c }_{ i }\left( { x }_{ ik }-{ \hat { \overline { x }  }  }_{ { w }_{ i }k } \right) \qquad (5.3)
$$

であり、(5.1)式のパラメータを最大部分尤度推定量$\hat {\beta}$で置き換えたものから求められる。ここで、
$$
{ \hat { \overline { x }  }  }_{ { w }_{ i }k }=\frac { \sum _{ j\in R\left( t_{ i } \right)  }{ { x }_{ jk }{ e }^{ { x }_{ j }\hat { \beta  }  } }  }{ \sum _{ j\in R\left( t_{ i } \right)  }{ { e }^{ { x }_{ j }\hat { \beta  }  } }  } 
$$
はリスク集合で条件付けた共変量の条件付き平均の推定量である。

 - パラメータの最大部分尤度推定量は、(5.1)式を0とおいた方程式の解なので、Schoefeld残差の和は0になる
 - 打ち切りを受けたデータのSchoenfeld残差は$c_{i} = 0$なので0となる

### スケール化Schoenfeld残差(scaled Schoenfeld residuals)
スケール化Schoenfeld残差は標準化変量と似ており、Schoenfeld残差の期待値が分散に依存しないようにスケールを調整したものである。

ここで、$i$番目のデータの$p$次元のSchoenfeld残差ベクトルを
$$
{\hat { r }}_{i}^{\prime} = \left( \hat {r}_{i1}, \hat {r}_{i2}, \cdots, \hat {r}_{ip} \right)
$$
とする。ただし、$\hat {r}_{ik}$は(5.3)式の推定量であるため、$c_{i}$のとき$\hat {r}_{ik}$は欠測になる。$i$番目のデータにおけるSchoenfeld残差ベクトルに対する$p \times p$次元の共分散行列の推定量を$\widehat { Var }\left( \hat {r}_{i}^{\ast} \right)$とし、$c_{i}=0$のときは共分散行列の推定量も欠測とする。

> $$
{ \hat { r }  }_{ i }^{ \ast  }={ \left[ \widehat { Var } \left( { \hat { r }  }_{ i } \right)  \right]  }^{ -1 }{ \hat { r }  }_{ i } \qquad (5.4)
$$

このとき、$i$番目のデータに対する共分散行列$\widehat {Var}\left( {\hat {r}}_{i} \right)$の対角要素は、
$$
\widehat { Var } \left( { \hat { r }  }_{ i } \right) _{ kk }=\sum _{ j\in R\left( t_{ i } \right)  }{ { \hat { w }  }_{ ij }\left( x_{ jk }-{ \hat { \overline { x }  }  }_{ w_{ i }k } \right) ^{ 2 } } 
$$
であり、非対角要素は、
$$
\widehat { Var } \left( { \hat { r }  }_{ i } \right) _{ kl }=\sum _{ j\in R\left( t_{ i } \right)  }{ { \hat { w }  }_{ ij }\left( x_{ jk }-{ \hat { \overline { x }  }  }_{ w_{ i }k } \right) \left( x_{ jl }-{ \hat { \overline { x }  }  }_{ w_{ i }l } \right)  } 
$$
である。ここで、重みは、
$$
{\hat {w}}_{ij} = \frac { e^{x_{j}\hat {\beta}} }{ \sum _{l \in R\left( t_{i} \right)}{ e^{x_{l}\hat {\beta}} } }
$$
である。

> Grambsch and Therneau(1994)によるスケール化Schoenfeld残差に対する近似計算方法:
> 
> $$
{ \left[ \widehat { Var } \left( { \hat { r }  }_{ i } \right)  \right]  }^{ -1 }\cong m\widehat { Var } \left( \hat { \beta  }  \right) \\
{\hat {r}}_{i}^{\ast} = m\widehat { Var }\left( \hat {\beta} \right) {\hat {r}}_{i} \qquad (5.5)
$$
>
> - $m$: イベント数
> - 共分散$\widehat { Var }\left( {\hat {r}}_{i} \right)$が時間によらずほとんど一定の値になるという性質に基づく方法
> - 異なるリスク集合に対して共変量の分布があまり変わらないという条件のもとで成り立つことが知られている

In [1]:
def calc_schoenfeld_residuals(df, beta, duration_col, event_col=None, strata=None):
    """
    Schoenfeld残差を計算
    """
    df = df.copy()

    # Sort on time
    df = df.sort_values(by=duration_col)

    n, p = df.shape
    self.strata = coalesce(strata, self.strata)
    if self.strata is not None:
        original_index = df.index.copy()
        df = df.set_index(self.strata)

    # Extract time and event
    T = df[duration_col]
    del df[duration_col]
    if event_col is None:
        E = pd.Series(np.ones(df.shape[0]), index=df.index)
    else:
        E = df[event_col]
        del df[event_col]

    self._check_values(df, E)
    df = df.astype(float)

    schoenfeld_residuals_arr = np.zeros((n, p), dtype=float)
    for i, (ti, ei) in reversed(list(enumerate(zip(T.values, E.values)))):
        xi = df.iloc[i]
        print(xi)

In [10]:
whas500_df.head()

,age,gender,hr,sysbp,diasbp,bmi,cvd,afb,sho,chf,...,miord,mitype,year,admitdate,disdate,fdate,los,dstat,lenfol,fstat
1,83,0,89,152,78,25.54051,1,1,0,0,...,1,0,1,01/13/1997,01/18/1997,12/31/2002,5,0,2178,0
2,49,0,84,120,60,24.02398,1,0,0,0,...,0,1,1,01/19/1997,01/24/1997,12/31/2002,5,0,2172,0
3,70,1,83,147,88,22.14290,0,0,0,0,...,0,1,1,01/01/1997,01/06/1997,12/31/2002,5,0,2190,0
4,70,0,65,123,76,26.63187,1,0,0,1,...,0,1,1,02/17/1997,02/27/1997,12/11/1997,10,0,297,1
5,70,0,63,135,85,24.41255,1,0,0,0,...,0,1,1,03/01/1997,03/07/1997,12/31/2002,6,0,2131,0


In [11]:
whas500_df.columns

Index(['age', 'gender', 'hr', 'sysbp', 'diasbp', 'bmi', 'cvd', 'afb', 'sho',
       'chf', 'av3', 'miord', 'mitype', 'year', 'admitdate', 'disdate',
       'fdate', 'los', 'dstat', 'lenfol', 'fstat'],
      dtype='object')

In [ ]:
from lifelines import CoxPHFitter

x_cols = ['age', 'gender']
y_cols = ['lenfol', 'fstat']

cox_ph_model = CoxPHFitter()
cox_ph_model.fit(whas500_df[x_cols + y_cols], '')

### マルチンゲール残差(martingale residual)
マルチンゲール残差は、計数過程による残差の定式化によってられた値。以下では、

 - 共変量(特徴量)ベクトル$x$をもつ1人の観察対象者を時間"0"から追跡し、興味のあるイベントが死亡であるという状況を想定する
 - イベントが必ず1回しか起こらない結果変数を扱う

とする。計数過程を用いて比例ハザードモデルを表現すると、時間$t$でイベントが発生したかどうかの"計数"に対する線形モデル($y = \beta_{0} + \beta_{1}x + \varepsilon$)のような形になる。
$$
N\left(t\right) = \Lambda \left( t,x,\beta \right) + M\left(t\right) \qquad (5.6)
$$


 - $N\left(t\right)$: ”計数”を表し、モデルの観測値部分に相当する。
 $$
 N\left( t \right) =\begin{cases} 0\quad t<追跡終了時 \\ 1\quad t\ge 追跡終了時 \end{cases}
 $$
 - $\Lambda \left( t,x,\beta \right)$: モデルの"系統成分"(線形回帰式でいうと$\beta_{0} + \beta_{1}x$)。詳しい導出過程はここでは省略するが、比例ハザードモデルの系統成分は、観察対象者の追跡終了時までは時点$t$における累積ハザードに等しく、観察対象者の追跡終了時より後では0である。
 $$
 \Lambda \left( t,x,\beta \right) = H\left(t,x,\beta\right)\\
 \Lambda \left( t,x,\beta  \right) =\begin{cases} \begin{matrix} { e }^{ x\beta  }{ H }_{ 0 }\left( t \right)  & t\le 追跡終了時 \\ 0 & t>追跡終了時 \end{matrix} \end{cases}
 $$
 - $M\left(t\right)$: モデルの"誤差成分"(線形回帰式でいうと$\varepsilon$)
     - マルチーンゲールという確率過程の性質をもつ
     - 期待値が0になる

> **マルチンゲール残差**<br>
> (5.6)式を変形すると、$M\left(t\right)$を”残差”のような形式で表すことができる。
> $$
M\left(t\right) = N\left(t\right) - \Lambda \left( t,x,\beta \right) \qquad (5.7)
$$
> 理論的には、マルチンゲール残差は各時間$t$で値をもつが、残差を求めるときは、実用上、**<font color="red">追跡終了時の値が利用される</font>**。<br>
> 観察対象者$i$における追跡終了時のマルチーンゲール残差: 
> $$
M\left(t_{i}\right) = N\left(t_{i}\right) -  H\left( t_{i},x_{i},\beta \right) \qquad (5.8)
$$

### (修正)Cox-Snell残差
追跡終了時の$N\left(t\right)$は1(イベント発生)か0(打ち切り)であるので$N\left(t_{i}\right) = c_{i}$となる。$H\left(t_{i}, x_{i}, \beta\right)$を$\hat {H}\left( t_{i},x_{i},\hat {\beta} \right)$で置き換えたマルチンゲール残差の推定量は以下である。
$$
\hat {M}\left(t_{i}\right) = c_{i} - \hat {H}\left( t_{i},x_{i},\hat {\beta} \right) \qquad (5.9)
$$

 - この残差は、一般的に用いられる残差の形式(結果変数の観測値と予測値の差)に一致する

### スコア残差(score residual)
次に示す残差は、(5.1)式の偏導関数を少し変形することでえられる。$k$番目の共変量のスコア関数は、
$$
\frac { \partial L_{p}\left(\beta\right) }{ \partial {\beta}_{k} } = \sum _{i=1}^{n}{L_{ik}} \qquad (5.13)
$$
である。

(5.13)式のうち、$i$番目の観察対象者の$k$番目の共変量(特徴量)に対するスコア残差は、
$$
L_{ik} = \sum _{j=1}^{n}{\left( x_{ik} - {\bar {x}}_{w_{j}k} \right)dM_{i}\left(t_{j}\right)} \qquad (5.14)
$$
である。

 - ${\bar {x}}_{w_{j}k}$: 時点$t_{j}$において(6.2)式から得られる
 - $d{M}_{i}\left( t_{j} \right)$: 時点$t_{j}$における$i$番目の観察対象者のマルチンゲール残差の変化量
 $$
 d{M}_{i}\left( t_{j} \right) = d{N}_{i}\left( t_{j} \right) - Y_{i}\left( t_{j} \right)e^{x_{i}\beta}h_{0}\left( t_{j} \right) \qquad (5.15)
 $$
     - $d{N}_{i}\left( t_{j} \right)$: 時点$t_{j}$における$i$番目の観察対象者の計数関数の変化量。打ち切りのデータは常に0となり、そうでないデータについては実際に観測された生存時間の時点でのみ1であり、それ以外の時点では0となる。
     - $Y_{i}\left( t_{j} \right)$: この関数は、リスク集合過程(at risk process)と呼ばれ、
     $$
     Y_{ i }\left( t_{ j } \right) =\begin{cases} 1,\quad t_{ j }\le t_{ i } \\ 0,\quad t_{ j }>t_{ i } \end{cases}
     $$
     で定義される
     - $h_{0}\left( t_{j} \right)$: 時点$t_{j}$における(5.10)式の値


以上の式を代入し整理すると、スコア残差の推定量が得られる。
$$
\hat { L } _{ ik }=c_{ i }\cdot (x_{ ik }-{ \hat { \overline { x }  }  }_{ w_{ i }k })-x_{ ik }\cdot \hat { H } \left( t_{ i },x_{ i },\hat { \beta  }  \right) +e^{ x_{ i }\hat { \beta  }  }\sum _{ t_{ j }\le t_{ i } }{ { \hat { \overline { x }  }  }_{ w_{ j }k }\frac { { c }_{ j } }{ \sum _{ l\in R\left( t_{ j } \right)  }{ { e }^{ { x }_{ l }\beta  } }  }  }   \qquad (5.16)
$$
さらに$i$番目の観察対象者に対する$p$次元のスコア残差ベクトルは、
$$
{\hat {L}}_{i}^{\prime} = \left( {\hat {L}}_{i1}, {\hat {L}}_{i2}, \cdots, {\hat {L}}_{ip} \right) \qquad (5.17)
$$
で定義できる。


### スケール化スコア残差(scaled score residual)
(5.17)式のスコア残差のスケールを調整したものがスケール化スコア残差である。
$$
{\hat {L}}_{i}^{\ast} = \widehat {Var}\left(\hat {\beta}\right){\hat {L}}_{i} \qquad (5.18)
$$
$\widehat {Var}\left( \hat {\beta} \right)$は、パラメータベクトルをその推定量で置き換えたの共分散行列の推定量

---
## 2. 比例ハザード性の仮定の評価

---
## 3. 比例ハザードモデルにおける回帰診断
モデル評価のうえでもう1つ重要なことは、次のようなデータが存在するかどうかを特定するために、回帰診断統計量(regression diagnostic statistics)を用いて診断を行うことである。

 1. **他と極端に乖離した値の共変量をもつ観察対象者**
 2. **パラメータ推定において、過度な影響を与えているデータ**
 3. **モデルの当てはめにおいて、過度な影響を与えているデータ**


> **<font color="blue">てこ比</font>**(leverage): 各観察対象者の共変量(特徴量)の値の”極端な乖離”を測る診断統計量
>
> - てこ比はある意味で共変量に対する残差と解釈することが可。線形回帰モデルやロジスティック回帰モデルにおいて、てこ比は、**共変量(特徴量)全体の平均値と各観察対象者の共変量(特徴量)の値との距離として求められる**。→$(x - \bar {x})^{2}$

(5.16)式および(5.17)式で定義したスコア残差は、共変量の値$x_{ik}$と各時点のリスク集合における共変量の平均${\hat {\overline {x}}}_{w_{j}k}$の距離に対する重み付き平均であり、比例ハザードモデルの影響度診断の重要な一部分を担っている。スコア残差は、連続型共変量において、共変量の値が平均から離れるほど残差の値が大きくなるという線形回帰モデルのてこ比と同じ性質をもつ。

上記の定義により、**<font color="red">てこ比の値が大きい観測対象者は、パラメータ推定値に大きな影響を与える恐れがある</font>**。比例ハザードモデルにおいて影響度を診断するためには、線形回帰モデルやロジスティック回帰モデルにおけるCookの距離に類似した統計量が必要となる。

> **<font color="blue">Cook's Distance</font>**<br>
> Cook距離の意義は、ある観察対象者をデータから除いた場合のパラメータ推定量の変化量を近似し、簡単に計算可能な統計量を与えることにある。
> $$
\Delta \hat {\beta}_{ki} = \hat {\beta}_{k} - \hat {\beta}_{k(-i)} \qquad (5.23)
$$
>
> - $\hat {\beta}_{k}$: $n$人の観察対象者すべてを用いて計算されたパラメータの最大部分尤度推定量
> - $\hat {\beta}_{k(-i)}$: $i$番目の被験者を除いたデータを用いて計算されたパラメータの最大部分尤度推定量

<br>

> **<font color="blue">Cain and Lange</font>**(1984)による(5.23)式の近似推定量<br>
> パラメータ推定値の変化を表す$k$次元ベクトルが
> $$
\Delta \hat {\beta}_{i} = \left(\hat {\beta} - \hat {\beta}_{(-i)}\right) \cong \widehat {Var}\left( \hat {\beta} \right)\hat {L}_{i} \qquad (6.24)
$$
> となることを示した。※この近似式はスケール化スコア残差と呼ばれる。
>
> - $\hat {L}_{i}$: (5.17)式のスコア残差ベクトル
> - $\widehat {Var}\left( \hat {\beta} \right)$: パラメータベクトルを推定値で置き換えた共分散行列の推定値

線形回帰回帰モデルやロジスティック回帰モデルにおいては、Cook距離は、パラメータ推定量全体に対する被験者の影響度を要約した統計量を与えるために用いられる場合がある。全体に対する影響度の指標は、
$$
\left( \hat {\beta} - \hat {\beta}_{(-i)} \right)^{\prime}\left[ \widehat {Var}\left( \hat {\beta} \right) \right]^{-1}\left( \hat {\beta} - \hat {\beta}_{(-i)} \right)
$$
で定義されており、(5.24)式を用いて、
$$
{ld}_{i} = \left( \Delta \hat { \beta  } _{ (i) } \right) ^{ \prime  }\left[ \widehat { Var } \left( \hat { \beta  }  \right)  \right] ^{ -1 }\left( \Delta \hat { \beta  } _{ (i) } \right) \\
\cong \left( { \hat { L }  }_{ i } \right) ^{ \prime  }\left[ \widehat { Var } \left( \hat { \beta  }  \right)  \right] \left[ \widehat { Var } \left( \hat { \beta  }  \right)  \right] ^{ -1 }\left[ \widehat { Var } \left( \hat { \beta  }  \right)  \right] \left( { \hat { L }  }_{ i } \right) 
$$
と近似することができる。したがって
$$
ld_{i} \cong \left( { \hat { L }  }_{ i } \right) ^{ \prime  }\left[ \widehat { Var } \left( \hat { \beta  }  \right)  \right] \left( { \hat { L }  }_{ i } \right) \qquad (5.25)
$$
となる。

さらに、Pettitt and Bin Daud (1989)は、(5.25)式の統計量は$i$番目の被験者を除いたときの対数部分尤度の変化量の近似であることを示した。このような背景から、この統計量は**<font color="blue">尤度変位統計量</font>**(likelihood displacement statistic)と呼ばれる。そのため、(5.25)式では$ld$という表記が用いられており、さらに、
$$
ld_{i} \cong 2 \left[ L_{p}\left(\hat {\beta}\right) - L_{p}\left( \hat {\beta}_{(-i)} \right) \right] \qquad (5.26)
$$
という近似が成立する。(5.25)式の$ld_{i}$とマルチンゲール残差の散布図は、影響力の大きい観察対象者を検討するうえで、最も理にかなっていると考えられる。

---
## 4. モデル適合度

### May and Hosmer(1998)によるモデル適合度検定
ロジスティック回帰モデルのHosmer–Lemesho 検定に近い発想のモデル適合度検定。データを$G$個のグループに分割し、各グループ内で観測値と期待値との乖離を検定する。ただし、ロジスティック回帰モデルのように全体の要約はできないので、全体の要約を行う場合は、グループ変数を共変量として追加したモデルの部分尤度比検定などを行う必要がある。 また、May&Hosmer(2004) シミュレーションから、経験的に良いグループ数
$$
G = \max {\left\{ 2, \min {\left\{ 10, {イベント数}/{40} \right\}} \right\}}
$$
が与えられている。

 1. 推定済みモデルからマルチンゲール残差とリスクスコアを求める
     - マルチンゲール残差: (5.9)式
     - リスクスコア: $x_{i}\hat {\beta}$
 2. リスクスコアを並べ替え、リスクスコアの$G$分位点に基づく$G$つのグループを作成する
 3. 各グループの観察対象者の打ち切りの指示変数$c_{i}$を合計することで、各グループの観測イベント数を算出する
 4. 打ち切りの指示変数$c_{i}$とマルチンゲール残差の差としてモデルの累積ハザードを求める
 5. グループごとに各観察対象者の累積ハザードを合計して、各グループの期待イベント数を算出する

### 比例ハザード用の決定係数$R^{2}$
あらゆる回帰モデルで用いられる$R^{2}$のような指標をいくつか示す。

**Nagelkerke**(1991)
>$$
{R_{p}}^{2} = 1 - \left\{ exp {\left[ \frac {2}{n} \left(\log {L_{0}} - \log {L_{p}} \right) \right]} \right\}
$$
>
> - $n$: データ数(打ち切りデータ、非打ち切りデータ)
> - $L_{p}$: $p$個の共変量をもつモデルに対する対数部分尤度
> - $L_{0}$: 共変量を持たないモデルに対する対数部分尤度

**O'Quigley et al.**(2005)
${R_{p}}^{2}$の拡張版で、${R_{p}}^{2}$の式の$n$を、観測されたイベント数$m$に置き換えたものである。そのため、**打ち切り割合にそれほど依存しない指標**である。
> $$
{R_{p,e}}^{2} = 1 - \left\{ exp {\left[ \frac {2}{m}\left( \log {L_{0}} - \log {L_{p}} \right) \right]} \right\}
$$
> "ランダム性を記述する指標(measure of explained randomness)"と呼ばれる。
> 
> - $m$: 観測されたイベント数

**Royston**(2006)<br>
${R_{p}}^{2}$の拡張版で、${R_{p}}^{2}$の式の$n$を、観測されたイベント数$m$に置き換えたものである。そのため、打ち切り割合にそれほど依存しない指標である。

>$$
{ R_{ p,v } }^{ 2 }=\frac { { R_{ p,e } }^{ 2 } }{ { R_{ p,e } }^{ 2 }+\cfrac { \pi  }{ 6 } (1-{ R_{ p,e } }^{ 2 }) }
$$
> "説明できる変動を記述する指標"と呼ばれる。